# Generation of "Similar" Sequences Using The Recurrent Variational Auto Encoder

# Introduction

I'm doing a thing to generate similar protein sequences, eventually.

In [ ]:
# To Try:
# ! Break into training files
# ! with different seq lens. 
# ! Start with longer sequences.
import keras

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

from keras.layers import Bidirectional, Dense, Embedding
from keras.layers import Input, Lambda, LSTM, RepeatVector
from keras.layers import Flatten, TimeDistributed, Layer, BatchNormalization
from keras.layers import Activation, Dropout, Activation
from keras.layers.advanced_activations import ELU

from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam, RMSprop
from keras import objectives

from keras.models import Model, load_model

from keras import backend as K
from keras.utils import plot_model, get_file

import numpy as np
import pandas as pd

import os
from IPython.display import Image
import matplotlib.pyplot as plt

## Data

### Overview

We are working with a file that contains
sequences. Each sequence is on a new line.
Because of the way the sequences are pulled,
there are additional "-" characters used for
alignment. We will strip these out, and pad
the front of the sequences.

In [ ]:
'''
url = 'https://raw.githubusercontent.com/badriadhikari'
deepcon_path = 'DEEPCON/master/deepcon-covariance/test'
file = '16pkA0.aln'
filepath = os.path.join(url, deepcon_path, file)
#file = get_file(file, filepath)
file = '/home/das-hund/PycharmProjects/autoencoders/data/pdb_seqres.txt'

with open(file, 'r') as sequence_file:
    sequences = sequence_file.read() \
                             .replace('-', '') \
                             .split('\n')

sequence_count = len(sequences)
print(f'[+] {sequence_count} Sequences in {file}')
print(f'[+] Subset of Sequences:')
print('\n\n'.join(sequences[:5]))
'''

In [ ]:
# ! Appears to be the case that we have
# Sequences of varying length. May need
# to train long seqs first.
file = '/home/das-hund/PycharmProjects/autoencoders/data/pdb_seqres.txt'

sequences = []
iter_ = 0
with open(file, 'r') as sequence_file:
    for iter_, line in enumerate(sequence_file):
        if (iter_ + 1) % 2 == 0:
            sequences.append(line.strip())
    
sequence_count = len(sequences)
print(f'[+] {sequence_count} Sequences in {file}')
print(f'[+] Subset of Sequences:')
print('\n\n'.join(sequences[:5]))
print('\n\n'.join(sequences[-5:]))

### Tokenize

After splitting the texts by new lines, we
want to map the characters to integers. Note
that we use the `char_level` argument to 
tokenize characters instead of words. 

Fortunately, we don't need to limit the 
number of tokens. There aren't many amino
acids available. Worth noting, the last 
layer is currently one hot encoded. This
is **absolutely** worth optimizing; 
however, time is a factor. 

We will also pad the sequences. To start,
we also won't be limiting sequence length.
I'll do some simple analysis to get a
distribution of sequence length. This will
give us an idea of whether it's worth
limiting the length. 

#### Train the Tokenizer

In [ ]:
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(sequences)
word_to_index = tokenizer.word_index
index_to_word = {
    index: word 
    for word, index in word_to_index.items()
}

In [ ]:
sequences = tokenizer.texts_to_sequences(sequences)

#### Summary Stats 

In [ ]:
sequence_lengths = np.array([
    len(seq) for seq in sequences
])

max_seq_len = max(sequence_lengths)
word_count = len(word_to_index)

print(f'[+] Max Sequence Length: {max_seq_len}')
print(f'[+] Word Count: {word_count}')

%matplotlib inline
plt.hist(sequence_lengths)

# ToDo: Update the names of these params.
MAX_WORDS = word_count
MAX_SEQ_LEN = 256#max_seq_len

In [ ]:
print(f'[+] {len(word_to_index)} words found in {file}')

print('[+] Word Map Subset:\n{')
for acid, index in word_to_index.items():
    print('\t{}: {}'.format(acid, index))
print('}')

#### Data Subset

When training the RNNS, we will need to limit the
length of the data to a multiple of what our batch
size will be. In the future, we will also perform 
the train test split here. But, for now, we just
select a random subset of the data. This will serve
as our development set. 

In [ ]:
DATA_LEN = 486400
np.random.shuffle(sequences)
sequences = sequences[:DATA_LEN]

#### Pad Sequences

In [ ]:
padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQ_LEN)

In [ ]:
print(padded_sequences[0:5])
print(padded_sequences.shape)

In [ ]:
#padded_sequences = padded_sequences.reshape((512, 256, 1))

## Model Architecture

Note I'm not using the GLOVE embeddings here.
That would make it difficult to transition this
model to protein sequences. 

In [ ]:
from keras.layers.advanced_activations import ELU


def build_encoder(encoder_input, max_seq_len, 
                  latent_dim, intermediate_dim,
                  epsilon_std):
    h = Bidirectional(LSTM(
        intermediate_dim, return_sequences=True, name='lstm_encoding_one'
    ), merge_mode='concat', name='bidirectional_encoding_one')(encoder_input)
    h = Bidirectional(LSTM(
        intermediate_dim // 2, return_sequences=False, name='lstm_encoding_two'
    ), merge_mode='concat', name='bidirectional_encoding_two')(h)

    def sampling(args):
        z_mean_, z_log_var_ = args
        batch_size = K.shape(z_mean_)[0]
        epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0., stddev=epsilon_std)
        return z_mean_ + K.exp(z_log_var_ / 2) * epsilon
    
    z_mean = Dense(latent_dim, activation='linear', name='z_mean')(h)
    z_log_var = Dense(latent_dim, activation='linear', name='z_log_var')(h)
    
    def vae_loss(x, x_decoded_mean):
        x = K.flatten(x)
        x_decoded_mean = K.flatten(x_decoded_mean)
        xent_loss = max_seq_len * objectives.binary_crossentropy(x, x_decoded_mean)
        kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return xent_loss + kl_loss
    
    latent = Lambda(
        sampling, output_shape=(latent_dim,), name='latent'
    )([z_mean, z_log_var])
    
    return vae_loss, latent


def build_decoder(encoded_input, intermediate_dim,
                  token_count, max_seq_len):
    repeated_context = RepeatVector(
        max_seq_len, name='repeated_context'
    )(encoded_input)
    
    h = LSTM(
        intermediate_dim // 2, return_sequences=True, name='lstm_decoding_one'
    )(repeated_context)
    h = LSTM(
        intermediate_dim, return_sequences=True, name='lstm_decoding_two'
    )(h)
    
    decoded = TimeDistributed(Dense(
        token_count, activation='softmax', name='time_distributed_decoding'
    ), name='decoded_mean')(h)
    
    return decoded
    
    
def build_model(max_seq_len, embedding_dim, token_count,
                batch_size, intermediate_dim, 
                latent_dim, epsilon_std=0.1):
    # ENCODER
    encoder_input = Input(shape=(max_seq_len, 1), name='encoder_input')
    
    vae_loss, encoded = build_encoder(
        encoder_input=encoder_input, max_seq_len=max_seq_len,
        latent_dim=latent_dim, intermediate_dim=intermediate_dim,
        epsilon_std=epsilon_std
    )
    
    encoder = Model(encoder_input, encoded, name='encoder')
    
    # DECODER
    encoded_input = Input(shape=(latent_dim,), name='encoded_input')
    decoded = build_decoder(
        encoded_input=encoded_input, intermediate_dim=intermediate_dim,
        token_count=token_count, max_seq_len=max_seq_len
    )
    
    decoder = Model(encoded_input, decoded, name='decoder')
    
    # VAE
    vae = Model(
        encoder_input, 
        build_decoder(
            encoded_input=encoded, intermediate_dim=intermediate_dim,
            token_count=token_count, max_seq_len=max_seq_len
        ), 
        name='vae')
    vae.compile(
        optimizer='adam',
        loss=vae_loss,
        metrics=['accuracy']
    )
    
    return vae, encoder, decoder


In [ ]:
MAX_SEQUENCE_LENGTH = MAX_SEQ_LEN
EMBED_DIM = 4
WORD_COUNT = MAX_WORDS + 1
BATCH_SIZE = 512
STEPS_PER_EPOCH = DATA_LEN // BATCH_SIZE
INTERMEDIATE_DIM = 128
LATENT_DIM = 32
EPOCHS = 6
reload_model = True
        
vae, encoder, decoder = build_model(
    max_seq_len=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    token_count=WORD_COUNT,
    batch_size=BATCH_SIZE,
    intermediate_dim=INTERMEDIATE_DIM,
    latent_dim=LATENT_DIM,
    epsilon_std=0.1
)

if reload_model:
    vae.load_weights('old_bestmodel.weights.hdf5')

In [ ]:
plot_model(vae, show_shapes=True, show_layer_names=True, to_file='vae.png')
Image(retina=True, filename='vae.png')

In [ ]:
vae.summary()

In [ ]:
def complex_oh_encode(X_train, maxlen, num_words):
    temp = np.zeros((X_train.shape[0], maxlen, num_words))
    temp[
        np.expand_dims(
            np.arange(X_train.shape[0]), axis=0
        ).reshape(
            X_train.shape[0], 1
        ), np.repeat(
            np.array([np.arange(maxlen)]), X_train.shape[0], axis=0
        ), X_train
    ] = 1
    return temp

In [ ]:
# Will need this when we get the model working.
def batch_generator(X, batch_size, max_seq_len, num_words):
    indices = np.arange(len(X)) 
    batch=[]
    while True:
        np.random.shuffle(indices) 
        for i in indices:
            batch.append(i)
            if len(batch)==batch_size:
                train = X[batch].reshape((batch_size, max_seq_len, 1))
                yield train, complex_oh_encode(X[batch], maxlen=max_seq_len, num_words=num_words)
                batch=[]

In [18]:
# TODO: Add metrics for KL Div & Expectation
fit_model = True
fit_generator = True

if fit_model:
    checkpoint = ModelCheckpoint(
        'old_bestmodel.weights.hdf5', monitor='loss',
        verbose=1, save_best_only=True, mode='min'
    )
    ## change to monitor loss
    reduce_lr = ReduceLROnPlateau(
        monitor='loss', patience=2, 
        min_lr=0.0001, verbose=1
    )
    callbacks_list = [checkpoint, reduce_lr]
    if fit_generator:
        train_generator = batch_generator(
            X=padded_sequences, 
            batch_size=BATCH_SIZE,
            max_seq_len=MAX_SEQUENCE_LENGTH,
            num_words=WORD_COUNT
        )
        history = vae.fit_generator(
            train_generator, 
            steps_per_epoch=STEPS_PER_EPOCH, 
            epochs=EPOCHS,
            callbacks=callbacks_list
        )
    else:
        trainX = padded_sequences.reshape(
            (DATA_LEN, MAX_SEQ_LEN, 1)
        )
        trainY = complex_oh_encode(
            padded_sequences, 
            maxlen=MAX_SEQ_LEN, 
            num_words=WORD_COUNT
        )
        history = model.fit(
            trainX, trainY,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            callbacks=callbacks_list
        )

Instructions for updating:
Use tf.cast instead.
Epoch 1/6
572/950 [=================>............] - ETA: 7:28 - loss: 26.0177 - acc: 0.3823

KeyboardInterrupt: 

In [ ]:
predictors = padded_sequences.reshape((DATA_LEN, 256, 1))
preds = vae.predict(predictors[0:10])


In [ ]:
' '.join([
    index_to_word[i] if i != 0 else ''
    for i in padded_sequences[5]
]).strip()

In [ ]:
for i in range(9):
    pred_chars = [np.argmax(l) for l in preds[i]]
    print(' '.join([
        index_to_word[i] if i != 0 else '' 
        for i in pred_chars
    ]).strip())
    print()

In [ ]:
#vae.save('vae.h5')
encoder.save('encoder.h5')
decoder.save('decoder.h5')